In [ ]:
%pwd
%cd ../..

In [ ]:
import glob

import numpy as np
import pandas as pd

from deeplifting.problems import PROBLEMS_BY_NAME

In [ ]:
# Get the results from deeplifting
files = glob.glob('./results/results-2023-08-03-2*')
deeplifting_results = pd.read_parquet(files)

# Select specific columns for results
deeplifting_results = deeplifting_results[
    ['problem_name', 'algorithm', 'x1', 'x2', 'f', 'total_time', 'global_minimum']
]

# For now let's remove sine-envelope function because there is something
# going on there
mask = deeplifting_results['problem_name'] != 'sine_envelope'
deeplifting_results = deeplifting_results.loc[mask].reset_index(drop=True)
deeplifting_results['hits'] = (
    np.abs(deeplifting_results['global_minimum'] - deeplifting_results['f']) <= 1e-4
)
deeplifting_results['hits'] = deeplifting_results['hits'].astype(int)
deeplifting_results = deeplifting_results.rename(columns={'total_time': 'time'})
deeplifting_results = deeplifting_results.drop(columns=['global_minimum'])
deeplifting_results.head()

In [ ]:
deeplifting_results.groupby('problem_name')['hits'].mean()

In [ ]:
# Load in the comparisons
comparison_files = glob.glob('./results/algorithm-comparisons-*.parquet')
comparison_results = pd.read_parquet(comparison_files)

# # Omit sine envelope function
# mask = comparison_results['problem_name'] != 'sine_envelope'
# comparison_results = comparison_results.loc[mask].reset_index(drop=True)
problems = [
    'ackley',
    'ackley2',
    'adjiman',
    'alpine1',
    'alpine2',
    'bartels_conn',
    'beale',
    'bird',
    'bohachevsky3',
    'booth',
    'bukin_n4',
    'bukin_n6',
    'camel_3hump',
    'camel_6hump',
    'chung_reynolds',
    'cube',
    'cross_in_tray',
    'cross_leg_table',
    'drop_wave',
    'eggholder',
    'griewank',
    'holder_table',
    'levy',
    'levy_n13',
    'rastrigin',
    'schaffer_n2',
    'schaffer_n4',
    'schwefel',
    'shubert',
    'ex8_1_1',
    'kriging_peaks_red010',
    'mathopt6',
    'quantum',
    'rosenbrock',
    'xinsheyang_n2',
    'xinsheyang_n3',
    'layeb3',
    'layeb4',
    'layeb6',
    'layeb7',
    'layeb8',
    'layeb8',
    'layeb12',
]

comparison_results = comparison_results.loc[
    comparison_results['problem_name'].isin(problems)
]

In [ ]:
comparison_results['problem_name'].value_counts()

In [ ]:
results_df = comparison_results.copy()

In [ ]:
results_df.loc[results_df['problem_name'] == 'ackley3']

In [ ]:
aggregated_results_df = (
    results_df.groupby(['problem_name', 'algorithm'])
    .agg({'hits': 'mean'})
    .rename(columns={'hits': 'Success Rate'})
    .unstack()
    .transpose()
    .reorder_levels(['algorithm', None])
    .transpose()
)

In [ ]:
aggregated_results_df.mean().sort_values(ascending=False)

In [ ]:
aggregated_results_df

In [ ]:
aggregated_results_df.shape

In [ ]:
file = [
    './results/results-2023-08-2-layer-512-holder_table-0-particle-leaky_relu.parquet',
    './results/results-2023-08-2-layer-512-mathopt6-0-particle-leaky_relu.parquet',
    './results/results-2023-08-2-layer-schwefel-0-particle-leaky_relu.parquet',
    './results/results-2023-08-2-layer-shubert-0-particle-leaky_relu.parquet',
    './results/results-2023-08-2-layer-512-cross_leg_table-0-particle-leaky_relu.parquet',
    './results/results-2023-08-2-layer-128-sum-ackley2-0-particle-leaky_relu.parquet',
    './results/results-2023-08-2-layer-128-sum-booth-0-particle-leaky_relu.parquet',
    './results/results-2023-08-2-layer-128-sum-adjiman-0-particle-leaky_relu.parquet',
    './results/results-2023-08-2-layer-128-sum-bukin_n4-0-particle-leaky_relu.parquet',
    './results/results-2023-08-2-layer-128-sum-camel_3hump-0-particle-leaky_relu.parquet',
    './results/results-2023-08-2-layer-128-sum-alpine1-0-particle-leaky_relu.parquet',
    './results/results-2023-08-2-layer-128-sum-camel_6hump-0-particle-leaky_relu.parquet',
    './results/results-2023-08-2-layer-128-sum-chung_reynolds-0-particle-leaky_relu.parquet',
    './results/results-2023-08-2-layer-128-sum-cube-0-particle-leaky_relu.parquet',
    './results/results-2023-08-2-layer-128-sum-alpine2-0-particle-leaky_relu.parquet',
    './results/results-2023-08-2-layer-128-sum-bartels_conn-0-particle-leaky_relu.parquet',
    './results/results-2023-08-2-layer-128-sum-beale-0-particle-leaky_relu.parquet',
    './results/results-2023-08-2-layer-128-sum-bird-0-particle-leaky_relu.parquet',
    './results/results-2023-08-2-layer-128-sum-bohachevsky3-0-particle-leaky_relu.parquet',
    './results/results-2023-08-2-layer-128-sum-xinsheyang_n2-0-particle-leaky_relu.parquet',
    './results/results-2023-08-2-layer-128-sum-xinsheyang_n3-0-particle-leaky_relu.parquet',
    './results/results-2023-08-2-layer-128-sum-layeb12-0-particle-leaky_relu.parquet',
    './results/results-2023-08-2-layer-128-max-layeb3-0-particle-sine.parquet',
    './results/results-2023-08-2-layer-128-sum-layeb4-0-particle-leaky_relu.parquet',
]
df = pd.read_parquet(file)

In [ ]:
df

In [ ]:
df['hits'] = np.abs(df['global_minimum'] - df['f']) <= 1e-4

In [ ]:
df.groupby('problem_name')['hits'].agg(['mean', 'count'])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def xinsheyang02(x, y):
    numerator = np.abs(x) + np.abs(y)
    demoninator = np.exp(np.sin(x**2) + np.sin(y**2))
    return numerator / demoninator


x = np.linspace(-2 * np.pi, 2 * np.pi, 10000)
y = np.linspace(-2 * np.pi, 2 * np.pi, 10000)
X, Y = np.meshgrid(x, y)
Z = xinsheyang02(X, Y)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, Z, cmap='jet')

ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')
ax.set_title('3D Plot of Ackley Function')

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def crosslegtable(x, y):
    component1 = np.sin(x) * np.sin(y)
    component2 = np.exp(np.abs(100 - (np.sqrt(x**2 + y**2)) / np.pi))
    denominator = np.abs(component1 * component2) + 1
    return -1 / (denominator) ** 0.1


x = np.linspace(-0.1, 0.1, 20000)
y = np.linspace(-0.1, 0.1, 20000)
X, Y = np.meshgrid(x, y)
Z = crosslegtable(X, Y)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, Z, cmap='jet')

ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')
ax.set_title('3D Plot of  Function')

plt.show()

In [ ]:
crosslegtable(0, 0)

In [ ]:
problem = PROBLEMS_BY_NAME['damavandi']

In [ ]:
problem

In [ ]:
import numpy as np
import torch

In [ ]:
results = np.zeros((1, 1, 3))
trial = 0
f = problem['objective']

In [ ]:
values = [1, 10]
f(torch.tensor(values), results=results, trial=trial, version='pytorch')

In [ ]:
f(np.array(values), results=results, trial=trial, version='numpy')

In [ ]:
x = [0, 1, 2, 3, 4]

In [ ]:
x[slice(0, 2)]

In [ ]:
x1, x2 = 0, 2

r = x1**2 + x2**2
theta = np.arctan(x2 / x1)
component1 = (r - 10) / (5**0.5 * np.cos(theta))

In [ ]:
(r, theta, component1)

In [ ]:
(1 - np.abs(component1) ** 5) * (2 + component1)

In [ ]:
np.sqrt(5)

In [ ]:
5**0.5

In [ ]:
x1, x2 = 0, 0
numerator = np.abs(x1) + np.abs(x2)
demoninator = np.exp(np.sin(x1**2) + np.sin())